# processing lesson plan
pandas dataframe = title of study guide
dictionary?
big picture: ... ---> generate a question that asks if this has been accomplished
terms: {term: definition, ...} if defintion not provided generate them
lesson headers: {topic: content, ...}

# practice material
test (randomly shuffling generated quizzes OR finding online tests to link to)
quizzes (vocab or comprehension)
flashcards with vocab terms
link to other resources found on the internet (web scraping)
chatbot to explain concepts: icon on the side, similar to customer service

# other features
satifisifaction ratings and need more explainations at the end of each material to allow for further elaboration

Reading in the study guide
- study guide sections are seperated by newlines, where the text immediately follows the header
- convert study guide to text file
- read into python string
- split into a list
- make list contents into a dictionary where {topic: content}

!{sys.executable} -m pip install library

In [74]:
from pathlib import Path
import os
import re
import requests
import json
import sys
from getpass import getpass
import time
import random
from IPython.display import clear_output

In [2]:
!{sys.executable} -m pip install profanity-check

  Using cached scikit_learn-1.3.2-cp39-cp39-win_amd64.whl (9.3 MB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


You should consider upgrading via the 'c:\users\hiloo\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
from profanity_check import predict_prob

In [2]:
# chatbot api key
OPENROUTER_API_KEY = getpass()
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

 ········


Generating the practice materials
VOCABULARY
- content including ": " usually indicates vocab word, split on that to make processing easier
- if vocab word doesn't contain a definition, use webscraping to find it
type 1: vocab flashcards
- flashcards, class flashcard, display_term(), display_definition(), self_dictionary
type 2: vocab quiz
- generate questions based on vocab terms class vocab quiz, get_questions(), get_answers()
type 3: vocab explaination
- get_alternative_definition(), get_example() (example is use of word in a sentence)

COMPREHENSION
type 1: testing on concepts
- when would we use ____?
type 2: example problems
- class example problems, pull from online to find example questions get_examples() or generate_examples() from chatbot and accompanying answers

Links
- online quizzes
- online tests
- extra resource material

EXTRA FEATURES
type 1: elaboration
- is this incomplete question?
- get_explaination() asking chatbot to elaborate given a lack of understanding
type 2: disclaimer
- disclaimer about inaccuracy for math, etc.
switch to multiple languages

In [3]:
def get_topic_content(string):
    first_newline_index = string.find("\n")
    topic = string[0:first_newline_index].strip()
    content = string[first_newline_index:].strip()
    return (topic, content)

In [4]:
# getting chatbot response
# contextualizing prompt requests
context = "Imagine you are a K-12 teacher and enhancing student understanding through good, detailed explanations."
def generate_chatbot_response(prompt):
    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        data=json.dumps({
            "models": ["nousresearch/nous-capybara-7b", "mistralai/mistral-7b-instruct","huggingfaceh4/zephyr-7b-beta"], 
            "messages": [{"role": "user", "content": context + " " + prompt}],
            "route": 'fallback'
        }),
        headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}"}
    )
    return get_json_content(response.json())

In [5]:
def get_json_content(json):
    content = json['choices'][0]['message']['content'].strip()
    # cleaning up the content, ie. spaces
    return content

In [ ]:
def check_profanity(content):
    pp = predict_prob([content])
    if pp[0] > 0.70:
        return "This content is offensive."
    else:
        return content

In [6]:
class Word:
    def __init__(self, term, definition=None):
        self.term = term
        if definition==None:
            definition_prompt = "what is the definition of " + term + "?"
            self.definition = generate_chatbot_response(definition_prompt)
            time.sleep(2)
        else:
            self.definition = definition

    def get_term(self):
        return self.term

    def get_definition(self):
        return self.definition

    def get_alternative_definition(self):
        alt_definition_prompt = "Rephrase the definition of " + self.term + "."
        alt_defintion = generate_chatbot_response(alt_definition_prompt)
        time.sleep(2)
        print("Alternative Definition:", alt_defintion)

    def get_example_use(self):
        example = "Given its definition: " + self.definition + ", use " + self.term + " in a sentence."
        example_use = generate_chatbot_response(example)
        time.sleep(2)
        print("Using",self.term,"in a sentence:",example_use)

    def update_definition(self, new_def):
        self.definition = new_def

    def __str__(self):
        return f"{self.term}: {self.definition}"

In [86]:
class Question:
    def __init__(self, question, answer):
        self.question = question
        self.correct_answer = answer
        ia_prompt = 'Given that the correct answer to the "' + question + '" is: "' + answer + '", generate 3 wrong answers for the question.'
        ia = generate_chatbot_response(ia_prompt)
        time.sleep(2)
        self.answer_choices = [k[len('3. '):len(k)] for k in ia.split("\n")] + [answer]

    def show_question(self):
        print(self.question)

    def show_answer(self):
        print(self.correct_answer)

    def show_incorrect_answers(self):
        print("\n".join(self.incorrect_answers))

    def pick_answer(self):
        choice = input("Pick a, b, c, or d.").lower().strip() 
        answer = self.answer_choices[int(chr(ord(choice)-49))]
        if answer == self.correct_answer:
            print("You answered correctly!")
            return True
        else:
            print("You answered incorrectly. The correct answer is: " + self.correct_answer)
            eia_prompt = 'Given that the correct answer to the "' + self.question + '" is: "' + self.correct_answer + '", explain why students might pick the incorrect answer "' + answer +'".'
            eia = generate_chatbot_response(eia_prompt)
            time.sleep(2)
            print(eia)
            return False

    def __str__(self):
        random.shuffle(self.answer_choices)
        output = f"{self.question}\nAnswer choices:\na) {self.answer_choices[0]}\nb) {self.answer_choices[1]}\nc) {self.answer_choices[2]}\nd) {self.answer_choices[3]}"
        return output

qna_list = re.split("\n\d\.",qna)
    
    for i in range(len(qna_list)):
        n = qna_list[i]
        index = n.find("?")
        q = n[:index+1]
        a = n[index+3:]
        l.append(Question(q, a))
        time.sleep(2) # to avoid errors in chatbot generation
    
    return l

In [57]:
def get_comprehension_questions(context):
    qna_prompt = f'Generate 3 difficult, complex, comprehension-focused questions and their answers based on the text: "{context}" for students.'
    qna = generate_chatbot_response(qna_prompt)
    time.sleep(2)
    n2 = re.split("\n\n",qna) # questions, Answers:, answers
    q = re.split("\d\.\s",n2[0])[1:] # split up the 3 questions
    questions = [f"{i+1}. {k}" for i in range(3) for k in q] # adding back in the number in front
    answers = re.split("\n",n2[2]) # splitting up the answers
    return questions,answers

In [79]:
def pick_an_answer(question,answer,name):
    print(question)
    user = input("Select a, b, c, or d.")[0].upper()
    letter_answer = answer[len("1. "):][:1]
    if letter_answer==user:
        print(f"Great job! You got it correct, {name}.")
        return True
    else:
        print(f"{name}, that's incorrect. The correct answer is {answer[len('1. '):]}")
        return False

In [75]:
class Flashcard:
    def __init__(self, Word):
        self.word = Word

    # private method
    def __display_flashcard(self,word):
        out = [(word[i:i+39]) for i in range(0, len(word), 39)]
        # 8 rows of the flashcard
        rows = abs(8 - len(out))
        line = "|                                       |"
        if rows % 2 != 0:
            top_line_n = int(rows/2) + 1
            bottom_n = top_line_n
        else:
            top_line_n = int(rows/2)
            bottom_n = top_line_n
        print("_________________________________________")
        for i in range(top_line_n):
            print(line)
        for i in out:
            l = int((39 - len(i))/2)
            string = " " * l + i + " " * l
            if len(i) % 2 != 0:
                print("|" + string + "|")
            else:
                print("|" + string + " |")
        for i in range(bottom_n):
            print(line)
        print("|_______________________________________|")

    def display_word_side(self):
        self.__display_flashcard(self.word.get_term())

    def display_definition_side(self):
        self.__display_flashcard(self.word.get_definition())

In [64]:
def process_study_guide(filepath):
    # PROCESSING INPUTTED STUDY GUIDE
    text = Path(filepath).read_text(encoding='utf-8')
    text = text[len("\ufeff"):]
    
    # delete to protect sensitive information
    #if os.path.exists(filepath):
        #os.remove(filepath)
    #else:
        #print("File not found.")

    # study guide is in a specific format
    text_sections = [x for x in re.split('\n\n|\n \n', text)]
    title = text_sections.pop(0)
    
    text_dictionary = {}
    text_dictionary.update([get_topic_content(i) for i in text_sections])

    # getting keywords (content including ": " usually indicates vocab words)
    text_vocab_list = []
    for i in text_dictionary.values():
        if re.match(".+\s?:\s?.+", i) != None:
            text_vocab_list.extend(i.split("\n")) # does not account for vocab terms starting with a \n, fix later

    return text, title.strip(), text_dictionary, text_vocab_list

In [80]:
# USER INTERACTION
name = input("Hello user! I am your personal A.I. tutor. Please keep in mind that the results a chatbot generates are not always 100% accurate as you utilize this study resource. Before we get started, what would you like me to call you?")
print(f"It is nice to meet you, {name}! Please supply me the path of your file so I can generate practice materials for you.")
filepath = input("Enter the filepath here: ") # C:\\Users\\hiloo\\Downloads\\proofs edited .txt

sg_text, sg_title, ssg_topic_dict, sg_vocab = process_study_guide(filepath)

print(f"Thanks, {name}! Now, I am going to display a menu of choices for you to use to review the material you've given me concerning: {sg_title}. Let's get into it!")

# Clearing the Screen
time.sleep(3)
clear_output()

while True:
    choice = input("Choose from the menu:\nA) Vocabulary Flashcards\nB) Vocab Quiz\nC) Comprehension Questions\nD) Exit\nEnter your choice: ").upper()
    if choice == "A": # vocab flashcards
        print("You selected: Vocabulary Flashcards.")
        time.sleep(2)
        clear_output()
        card_n = len(sg_vocab)
        i = 0
        f = True
        while i < card_n:
            print(f"Showing {i+1} out of {card_n} words.")
            # parsing vocab words
            term = sg_vocab[i].split(": ")[0]
            definition = sg_vocab[i].split(": ")[1]
            word = Word(term, definition)
    
            # creating and displaying flashcards
            flashcard = Flashcard(word) 
            if f:
                flashcard.display_word_side()
                choice = input("F) Flip the flashcard\nN) Move to the next flashcard.\nP) Move to the previous flashcard\nEnter your choice: ")[0].upper()
                if choice == "F":
                    f = not f
                    clear_output()
                elif choice == "N":
                    i += 1
                    clear_output()
                elif choice == "P":
                    if i - 1 >= 0:
                        i -= 1
                    clear_output()
                else:
                    print(f"That's not a choice, {name}! Please select a valid option.")
                    time.sleep(2)
                    clear_output()
            else:
                flashcard.display_definition_side()
                choice = input("F) Flip the flashcard\nN) Move to the next flashcard.\nP) Move to the previous flashcard\nEnter your choice: ")[0].upper()
                if choice == "F":
                    f = not f
                    clear_output()
                elif choice == "N":
                    i += 1
                    f = True
                    clear_output()
                elif choice == "P":
                    if i - 1 >= 0:
                        i -= 1
                    clear_output()
                else:
                    print(f"That's not a choice, {name}! Please select a valid option.")
                    time.sleep(2)
                    clear_output()
        print(f"Great job, {name}. You completed the flashcard review! Returning to menu...")
        time.sleep(3)
        clear_output()
    elif choice == "B": # vocab quiz
        print("You selected: Vocabulary Quiz.")
        time.sleep(2)
        clear_output()
        i = 0
        while i < len(sg_vocab):
            term = sg_vocab[i].split(": ")[0]
            definition = sg_vocab[i].split(": ")[1]
            all_terms = [k.split(": ")[1] for k in sg_vocab]
            all_terms.pop(i)
            random.shuffle(all_terms)
            defs = all_terms[:3]
            defs.append(definition)
            random.shuffle(defs)
            answer = input(f"What is the definition of: {term}?\na) {defs[0]}\nb) {defs[1]}\nc) {defs[2]}\nd) {defs[3]}\nEnter a choice: ")[0].lower()
            n = int(chr(ord(answer)-49))
            if defs[n] == definition:
                print(f"You got it correct, {name}!")
                print("Moving onto the next question...")
                i += 1
                time.sleep(2)
                clear_output()
            else:
                print(f"That's incorrect! The correct definition is {definition}")
                print("Moving onto the next question...")
                i += 1
                time.sleep(2)
                clear_output()
        print(f"Great job, {name}. You completed the vocab quiz review! Returning to menu...")
        time.sleep(3)
        clear_output()
    elif choice == "C": # comprehension questions
        print("You selected: Comprehension Questions.")
        time.sleep(2)
        clear_output()
        print(f"Hello, {name}! I will now be giving you a comprehension quiz on {sg_title}. Please keep in mind this section contains A.I. generated material and therefore possible inaccuracies.")
        print("Analyzing text and generating questions...")
        clear_output(wait=True)
        cont = True
        while cont:
            q,a= get_comprehension_questions(sg_text)
            n_correct = 0
            for i in range(len(a)):
                val = pick_an_answer(q[i],a[i],name)
                if val:
                    n_correct += 1
                time.sleep(2)
                clear_output()
            if n_correct == 3:
                msg = f"You're an expert!"
            elif n_correct == 2:
                msg = f"Your score was nearly perfect."
            else:
                msg = f"Good try. Keep reviewing the material and your score will improve."
            print(f"Congratulations {name}! You completed the quiz on {sg_title}.\n{msg}\nYour score was {n_correct}/3 ({round(n_correct/3*100,2)}%).")
            c = input(f"You have a decision to make, {name}.\nA) Re-take the quiz\nB) Exit\nEnter your choice: ")[0].upper()
            if c == "A":
                print(f"Okay, {name}. Let's re-do this!")
                time.sleep(2)
                clear_output()
            elif c == "B":
                cont = False
                print("Returning to the menu...")
                time.sleep(2)
                clear_output()
            else:
                cont = False
                print(f"That's not a choice, {name}! Returning to the menu...")
                time.sleep(2)
                clear_output()
    elif choice == "D": # exit
        print(f"Aw, we're sad to see you go, {name}. Hopefully, you're leaving with a better understanding!")
        break
    else:
        print(f"That's not a choice, {name}! Please select a valid option.")

Choose from the menu:
A) Vocabulary Flashcards
B) Vocab Quiz
C) Comprehension Questions
D) Exit
Enter your choice:  d


Aw, we're sad to see you go, Jada. Hopefully, you're leaving with a better understanding!
